# MNIST Example with Data Logging in DataFed

## Import Libraries

In [3]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


## Builds the CNN

In [5]:
# Define the CNN architecture
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)  # Output layer for 10 classes (digits 0-9)
    
    def forward(self, x):
        # Apply convolutional layers with ReLU and max pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Flatten the output
        x = x.view(-1, 64 * 7 * 7)

        # Apply fully connected layers with ReLU and final output
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x


## Define transformations for data preprocessing

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize with mean and std of MNIST dataset
])


## Load the MNIST dataset


In [7]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 29539805.94it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 5651567.31it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

100%|██████████| 1648877/1648877 [00:00<00:00, 34208635.38it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4130643.71it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## Instantiate the model, loss function, and optimizer


In [10]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()  # Loss function for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer


## Training function


In [11]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()  # Set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        output = model(data)
        loss = criterion(output, target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')


## Testing function

In [12]:
def test(model, device, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            # Forward pass
            output = model(data)
            test_loss += criterion(output, target).item()  # Sum up the batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)\n')


## Instantiate the DataFed Configuration

In [11]:
%load_ext autoreload
%autoreload 2

from datafed_torchflow.datafed import DataFed

DataFed_path = "delete/a/g"

instance = DataFed(DataFed_path)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
instance.create_subfolder_if_not_exits()

In [13]:
instance.collection_id

'c/525610589'

In [27]:
# check if the sub-collection exists in DataFed
ls_resp = self.collectionItemsList(self.getRootColl)

trials_already_in_DataFed = []
for record in ls_resp[0].item:
    trials_already_in_DataFed.append(record.title)

In [1]:
from datafed_torchflow.pytorch import TorchLogger

DataFed_path = "/delete/this/path"

model = []

logger = TorchLogger(model, DataFed_path, verbose=True)

Unable to connect to pypi: <Fault -32500: 'RuntimeError: PyPI no longer supports the XMLRPC package_releases method. Use JSON or Simple API instead. See https://github.com/pypi/warehouse/issues/16642 and https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.'>
Success! You have been authenticated into DataFed as: u/jca318
Success! You have set up the Globus endpoint 7d7d5294-23aa-11ef-af02-21fa2ca908a5.


## Train Model

In [13]:
# Train and test the CNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

n_epochs = 5
for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312099
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.283328
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.116364
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.105909
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.096583
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.023996
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.190609
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.053661
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.034528
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.065710

Test set: Average loss: 0.0000, Accuracy: 9865/10000 (98.65%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.055813
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.239598
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.004509
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.011623
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.017995
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.022242
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.033130
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.023607
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.07232

In [ ]:

# Function to save the model
def save_model(model, filename="mnist_cnn.pth"):
    """
    Saves the model's state dictionary.

    Args:
        model: The PyTorch model to save.
        filename (str): The file path to save the model state dictionary.
    """
    torch.save(model.state_dict(), filename)
    print(f"Model saved to {filename}")

# Function to load the model
def load_model(model, filename="mnist_cnn.pth"):
    """
    Loads the model's state dictionary.

    Args:
        model: The PyTorch model to load the state dictionary into.
        filename (str): The file path to load the model state dictionary from.
    """
    model.load_state_dict(torch.load(filename))
    model.eval()  # Set the model to evaluation mode
    print(f"Model loaded from {filename}")



# Save the model after training
save_model(model, "mnist_cnn.pth")

# Load the model (optional)
# load_model(model, "mnist_cnn.pth")
